In [1]:
import json
import numpy as np

In [2]:
def json_loader(folder, filename):
    with open(folder+filename) as fh:
        file = json.load(fh)
    return file

questionnaire_loader = lambda name: json_loader("questionnaires/", name)

index = questionnaire_loader("index.json")
index

{'PHQ-9': {'filename': 'phq-9.json', 'topic': 'depression'},
 'GAD-7': {'filename': 'gad-7.json', 'topic': 'anxiety'},
 'ASRS-5': {'filename': 'asrs-5.json', 'topic': 'adhd'}}

In [3]:
def ask(question, scale):
    print(question)
    print("\n".join([f"{score} - {word}" for word, score in scale.items()]))
    print()
    possible_answers = set(scale.values())
    output = None
    while output is None:
        try:
            answer = int(input("Answer:"))
            if answer in possible_answers: output = answer
            else: raise(ValueError)
        except:
            print("Not a valid answer, please enter an integer from the list of possible responses.\n")
    return output

def evaluate_section(section, type="list"):
    prefix, questions, scale = section["prefix"], section["questions"], section["scale"]
    results = [ask(" ".join([prefix, q]), scale) for q in questions]
    match type:
        case "list": return results
        case "sum": return np.sum(results)

In [4]:
def PHQ9(filename="phq-9.json"):
    questionnaire = questionnaire_loader(filename)
    sections = questionnaire["sections"]
    scoring_functions = questionnaire["scoring"]
    
    #  SECTION 1 - Get responses to section 1 questions
    s1 = sections[0]
    s1_labels = {v: k for k, v in s1["scale"].items()}
    s1_responses = evaluate_section(s1)
    s1_total = np.sum(s1_responses)
    s1_scoring = scoring_functions[0]

    #  SECTION 2 - If any responses positive in section 1, complete section 2
    s2 = sections[1]
    s2_labels = {v: k for k, v in s2["scale"].items()}
    s2_response  = evaluate_section(s2, "sum") if any(s1_responses) else 0

    #  SEVERITY AND ACTION - Use scale to get severity of symptoms and recommended action
    severity, action = "", ""
    scoring_ranges = [range(a, b) for a, b in s1_scoring["ranges"]]
    severity_scale = [severity for severity in s1_scoring["severity"]]
    action_scale = [action for action in s1_scoring["action"]]
    for i, score_range in enumerate(scoring_ranges):
        if s1_total in score_range:
            severity = severity_scale[i]
            action = action_scale[i]
            break

    #  MDD CHECK - Check for Major Depressive Disorder, or other depressive syndromes
    mdd, other = False, False
    if s1_responses[0] >= 2 or s1_responses[1] >= 2:
        n_more_than_half = len([i for i in s1_responses if i >= 2])
        if n_more_than_half >= 5: mdd = True
        elif n_more_than_half >= 2: other = True

    #  PRINT RESULTS - Format results to be readable
    print(f"Depression Severity: {severity}")
    print(f"Recommended action: {action}")
    print(f"Functional health: {s2_labels[s2_response]}") 
    if mdd: print(f"! Major Depressive Disorder suggested")
    if other: print(f"! Major depressive disorder not suggested, but other depressive syndrome suggested")

In [5]:
def GAD7(filename="gad-7.json"):
    questionnaire = questionnaire_loader(filename)
    sections = questionnaire["sections"]
    scoring_functions = questionnaire["scoring"]
    
    #  SECTION 1 - Get responses to section 1 questions
    s1 = sections[0]
    s1_labels = {v: k for k, v in s1["scale"].items()}
    s1_responses = evaluate_section(s1)
    s1_total = np.sum(s1_responses)
    s1_scoring = scoring_functions[0]

    #  SECTION 2 - If any responses positive in section 1, complete section 2
    s2 = sections[1]
    s2_labels = {v: k for k, v in s2["scale"].items()}
    s2_response  = evaluate_section(s2, "sum") if any(s1_responses) else 0

    #  SEVERITY - Use scale to get severity of symptoms
    severity = ""
    scoring_ranges = [range(a, b) for a, b in s1_scoring["ranges"]]
    severity_scale = [severity for severity in s1_scoring["severity"]]
    for i, score_range in enumerate(scoring_ranges):
        if s1_total in score_range:
            severity = severity_scale[i]
            break

    #  PRINT RESULTS - Format results to be readable
    print(f"Anxiety Severity: {severity}")
    print(f"Functional health: {s2_labels[s2_response]}")

In [6]:
def ASRS5(filename="asrs-5.json"):
    questionnaire = questionnaire_loader(filename)
    sections = questionnaire["sections"]
    scoring_functions = questionnaire["scoring"]
    
    #  SECTION 1 - Get responses to section 1 questions
    s1 = sections[0]
    s1_labels = {v: k for k, v in s1["scale"].items()}
    s1_responses = evaluate_section(s1)
    s1_total = np.sum(s1_responses)
    s1_scoring = scoring_functions[0]

    #  SEVERITY - Use scale to check if user screens positive
    result = None
    scoring_ranges = [range(a, b) for a, b in s1_scoring["ranges"]]
    severity_scale = [outcome for outcome in s1_scoring["severity"]]
    for i, score_range in enumerate(scoring_ranges):
        if s1_total in score_range:
            result = severity_scale[i]
            break

    #  PRINT RESULTS - Format results to be readable
    print(f"ADHD Screening: {result}")